In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import cmath
import sdeint

ヒルベルト変換によってデータから位相を取り出し、位相モデルのパラメータを最尤推定。

次のモデルを仮定してパラメータ$\omega_j, b_{jk}, \sigma_i^2$を推定。
$$\dot{\phi}_j = \omega_j + \sum_k b_{jk}\sin(\phi_k-\phi_j) + \eta_j (t)$$
を仮定する。
ただし、$\eta_j$はガウシアンホワイトノイズ。$\langle \eta_j \rangle = 0, \langle \eta_i(t_1) \eta_j(t_2) \rangle = \delta_{ij}\sigma_i^2\delta(t_1-t_2)$

データ　x[j][t] ：j番目のデータの時刻tの値。

## データxからモデルパラメータを最尤推定
注意：ノイズ強度の推定は未実装。

In [93]:
aa0 = np.array([1,2,3])
print(aa0.reshape((3,1)) - aa0.reshape((1,3)))

[[ 0 -1 -2]
 [ 1  0 -1]
 [ 2  1  0]]


In [380]:
h = 0.01 #観測データの時間間隔
tau = 0.01 #データ生成の刻み幅
def MLE_phase_model(x):
    K = len(x) #振動子数
    T = len(x.T) #データ長
    #print(T)
    
    #xが位相データではない場合。ヒルベルト変換する。
    #analytic_signal = signal.hilbert(x)
    #phase = np.angle(analytic_signal) #-piからpiへジャンプがある。
    
    #xが位相データの場合
    analytic_signal = np.cos(x) + 1j*np.sin(x) 
    phase = x
    
    
    #Diff_phase_k[t]は、(i,j)成分が時刻tにおける振動子i,jの位相差(phi_j - phi_i)であるような行列。
    Diff_phase_k = np.array([phase.T[t].reshape((1,K)) - (phase.T[t]).reshape((K,1)) for t in range(T-1)]).reshape(T-1, K, K)
    Diff_phase_t = np.angle(analytic_signal.T[1:] / analytic_signal.T[:T-1])  #位相のジャンプがないように、解析信号の偏角の差をとることで求める。
    #print(Diff_phase_t)
    for j in range(K):
        A1 = (np.sum(np.sin(Diff_phase_k), axis=0)[j]).reshape((1, K))
        #print(A1)
        A2 = np.sum(np.array([np.dot(np.sin(Diff_phase_k[t][j]).reshape((K,1)), np.sin(Diff_phase_k[t][j]).reshape((1,K))) for t in range(T-1)]), axis=0)
        AA1 = np.append(np.array([T-1]).reshape((1,1)), A1).reshape((1,K+1))
        #print(AA1)
        A_with_zero = np.append(AA1,   np.append(A1, A2, axis=0).T, axis=0)
        #print("zero vector?", A_with_zero[j+1])
        A = np.delete(np.delete(A_with_zero, j+1, 0), j+1, 1) * h
        #print("A=", A)
        
        
        DeltaPhi = Diff_phase_t.T[j]
        bb = np.array([ np.sum (np.array([np.sin(Diff_phase_k[t][j][i]) * DeltaPhi[t] for t in range(T-1) ])) for i in range(K)])
        b_with_zero = np.append(np.sum(DeltaPhi).reshape((1,1)), bb.reshape((1,K)), axis=1).T
        
        #b_with_zero =  (1/h)*np.append(np.sum(Diff_phase_t.T[j]).reshape((1,1)),  np.sum( np.dot((Diff_phase_t.T[j]).reshape((T-1,1)), np.ones((1,K))).reshape((T-1,K)) * (np.sin(Diff_phase_k)[:,j,:]).reshape((T-1, K)), axis=0).reshape((1,K)), axis=1 ).T
        b =  np.delete(b_with_zero, j+1)
        #print("b=", b)
        
        x = np.linalg.solve(A,b)
        omega = x[0]
        bj = np.delete(x, 0)
        
        sigma_double = (1/(T*h)) * np.sum ( ((Diff_phase_t.T[j]).reshape((1,T-1)) - h* (omega + np.dot(bj.reshape((1,K-1)), np.sin(np.delete(Diff_phase_k[:,j,:], j, axis=1).reshape((K-1, T-1)))))  )**2 ) 
        
        #print(np.shape(((Diff_phase_t.T[j]).reshape((1,T-1)) -  (omega + np.dot(bj.reshape((1,K-1)), np.sin(np.delete(Diff_phase_k[:,j,:], j, axis=1).reshape((K-1, T-1)))))  )**2 ))
        
        print("固有周波数", j,  "=", omega)
        print("結合強度", j,"=", bj)
        print("ノイズ強度",j ,"=", np.sqrt(sigma_double))
        
        

## データ生成
### 最尤推定において仮定した位相モデル

In [407]:
num_node = 2 #振動子数
omega = np.array([6.28, 6.28]) #固有周波数
Connectivity_Matrix = np.array([[0,0.3], [0.2,0]]) 

sigma = 0.01


def G_whitenoise(x,t):
    return np.diag(np.ones(num_node)* sigma)
    
def phase_model(y, t): #tとyの順番
    sogosayo = np.diag(np.dot(np.sin(y.reshape((1, num_node)) - y.reshape((num_node, 1))), Connectivity_Matrix.T))
    return (omega +  sogosayo )


### データ生成

In [412]:
x_init= np.array([5.118909, 5.691091]) #初期値
T0 = 100
tspan0 = np.linspace(0.0, T0, int(T0/tau)+1)
result0 = sdeint.itoint(f=phase_model, G=G_whitenoise, y0=x_init, tspan=tspan0)


### 小林先生のコードで生成したデータ

In [413]:
result1 = np.loadtxt("Sin_Coupled/matsuki_test.dat")

In [414]:
result1 = np.delete(result1,0,  axis=1)

### 推定

In [415]:
MLE_phase_model((result0).T)

固有周波数 0 = 6.27913918053885
結合強度 0 = [0.29825362]
ノイズ強度 0 = 0.010042564096250253
固有周波数 1 = 6.280441084598153
結合強度 1 = [0.20759069]
ノイズ強度 1 = 0.010061511468480947
